In [1]:
"""
This example is from doi:10.1016/j.jcat.2007.05.011, Horn 2007
Ref 17 is doi:10.1016/j.cat.2006.05.008
Ref 18 is doi:10.1007/s10562-006-0117-8

surface to reactor volume ratio (assuming same active catalytic area) A/V, at 1600.0 m-1, from Ref 17
assume catalyst is a monolith, at 16.5 mm in diam, from Ref 17 & Ref 18
porosity is 0.81, from Ref 17
front heat sheild, cataylst, and back heat sheild is 10 mm long, from Ref 17
inlet mol flow at 0.208 mol/min

graphs are in:
exit conversion % v c/0 ratio
flow/mol/min v position(mm)
S at end oxidation zone/ % v C/O ratio
selectivity S, yield(Y)/% v C/O ratio
flow/ mol/min v position (z)/ mm

also had plots with flow/ mol/ min v C/O ratio w lines of thermodynamic equilibrium )ph = const)
"""
# load a bunch of stuff
import cantera as ct
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
plt.switch_backend('agg')  # needed for saving figures
import csv
from pydas.dassl import DASSL
import os
import rmgpy
import rmg
import re
import operator
import pandas as pd
import os
from collections import defaultdict
from collections import Counter

In [4]:
# this chemkin file is from the cti generated by rmg
gas = ct.Solution('/scratch/blais.ch/HAN/_10_HAN_RUN_20Apr2020/HAN/HAN_Runs/CatArea2Vol_1e3/cantera/chem_annotated.cti', 'gas')
surf = ct.Interface('/scratch/blais.ch/HAN/_10_HAN_RUN_20Apr2020/HAN/HAN_Runs/CatArea2Vol_1e3/cantera/chem_annotated.cti', 'surface1', [gas])

print("This mechanism contains {} gas reactions and {} surface reactions".format(gas.n_reactions, surf.n_reactions))

# unit conversion factors to SI
mm = 0.001
ms = mm
minute = 60.0

This mechanism contains 1563 gas reactions and 156 surface reactions


In [5]:
#######################################################################
# Input Parameters
#######################################################################
t_in = 400  # K - in the paper, it was ~698.15K at the start of the cat surface and ~373.15 for the gas inlet temp
t_cat = t_in
length = 70 * mm  # Reactor length- m
diam = 16.5*mm  # Reactor diameter - in m
area = (diam/2.0)**2*np.pi  # Reactor cross section area (area of tube) in m^2
porosity = 0.81  # Monolith channel porosity, from Horn ref 17 sec 2.2.2
# cat_area_per_vol = 1600.  # Catalyst particle surface area per unit volume in m-1
cat_area_per_vol = 10000  # I made this up, in m-1. 4500 is lowest that "work" for all base
flow_rate = 4.7  # slpm
flow_rate = flow_rate*.001/60  # m^3/s
tot_flow = 0.208  # from Horn 2007, constant inlet flow rate in mol/min, equivalent to 4.7 slpm
velocity = flow_rate/area  # m/s

# The PFR will be simulated by a chain of 'NReactors' stirred reactors.
NReactors = 7001

on_catalyst = 1000  # catalyst length 10mm, but it doesn't say where.  let's guess at 1 cm?
off_catalyst = 2000
dt = 1.0

reactor_len = length/(NReactors-1)
rvol = area * reactor_len * porosity

# catalyst area in one reactor
cat_area = cat_area_per_vol * rvol

# input mol fractions
ratio = 1.
fo2 = 1 / (2. * ratio + 1 + 79 / 21)
fch4 = 2 * fo2 * ratio
far = 79 * fo2 / 21
X = str('CH4(2):' + str(fch4) + ', O2(3):' + str(fo2))
gas.TPX = 273.15, ct.one_atm, X  # need to initialize mass flow rate at STP

mass_flow_rate = flow_rate * gas.density_mass
gas.TPX = t_in, ct.one_atm, X
temp_cat = t_in
surf.TP = t_in, ct.one_atm
surf.coverages = 'X(1):1.0'
gas.set_multiplier(1.0)

TDY = gas.TDY
cov = surf.coverages

# create a new reactor
gas.TDY = TDY
r = ct.IdealGasReactor(gas)
r.volume = rvol

# create a reservoir to represent the reactor immediately upstream. Note
# that the gas object is set already to the state of the upstream reactor
upstream = ct.Reservoir(gas, name='upstream')

# create a reservoir for the reactor to exhaust into. The composition of
# this reservoir is irrelevant.
downstream = ct.Reservoir(gas, name='downstream')

# Add the reacting surface to the reactor. The area is set to the desired
# catalyst area in the reactor.
rsurf = ct.ReactorSurface(surf, r, A=cat_area)

# The mass flow rate into the reactor will be fixed by using a
# MassFlowController object.
m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

# We need an outlet to the downstream reservoir. This will determine the
# pressure in the reactor. The value of K will only affect the transient
# pressure difference.
v = ct.PressureController(r, downstream, master=m, K=1e-5)

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 12

# set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-10
sim.atol = 1.0e-20

gas_names = gas.species_names
surf_names = surf.species_names

# dictionary of molecular weights
mw = {'H': 1.008, 'O': 15.999, 'C': 12.011}

# where the data will be stored
integration_flux_data = defaultdict(float)

def get_composition(species):
    """
    gets the composition of a species
    """
    try:
        index = gas.species_index(species)
        sp = gas.species(index)
        return sp.composition
    except:
        index = surf.species_index(species)
        sp = surf.species(index)
        return sp.composition


surf.set_multiplier(0.0)  # no surface reactions until the gauze
for n in range(NReactors):
    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = r.thermo.TDY
    upstream.syncState()
    if n == on_catalyst:
        surf.set_multiplier(1.0)
    if n == off_catalyst:
        surf.set_multiplier(0.0)
    sim.reinitialize()
    sim.advance_to_steady_state()

    # make reaction diagrams
    out_dir = 'rxnpath'
    os.path.exists(out_dir) or os.makedirs(out_dir)

    # getting the entire net rates of the system
    elements = ['H', 'O', 'C', 'N']
    temp_data = defaultdict(float)
    for element in elements:
        diagrams = [ct.ReactionPathDiagram(surf, element), ct.ReactionPathDiagram(gas, element)]
        for i in range(len(diagrams)):
            data = diagrams[i].get_data()
            split_data = data.split("\n")

            for line in split_data[2:]:  # skip the first 2 lines
                if len(line.split()) == 0:  # skip the empty line at the end
                    continue

                s1, s2, fwd, rev = line.split()
                net = float(fwd) + float(rev)

                # if nothing interesting, continue
                if net == 0.0:
                    continue

                # find the count of element in the species
                s1_composition = get_composition(s1)

                # multiply by atomic mass of the element
                net = float(fwd) * mw[element] * s1_composition[element]

                # for surface reactions, multiply by the catalyst area per volume in a reactor
                if i == 0:
                    net = net * cat_area

                flux_pair = (s1, s2)
                if (flux_pair or flux_pair[::-1]) in list(temp_data.keys()):
                    continue
                temp_data[flux_pair] += net

    # adding the temp data to the real dictionary
    integration_flux_data = Counter(integration_flux_data) + Counter(temp_data)

with open("integration_flux_data.txt", 'w') as f:
    for (sp1, sp2), flux in integration_flux_data.items():
        f.write("{} {} {}\n".format(sp1, sp2, flux))

CanteraError: 
***********************************************************************
CanteraError thrown by Phase::setMoleFractionsByName:
Unknown species 'Ar'
***********************************************************************
